## Multi Layer Perceptron using Grid Search K fold 

In [1]:
import pandas as pd
from cbfv.composition import generate_features
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GroupKFold

from keras.models import Sequential, Model
from keras.layers import Dense, Input

from keras.wrappers.scikit_learn import KerasRegressor

import pandas as pd
from cbfv.composition import generate_features


import tensorflow
from sklearn.model_selection import GridSearchCV



2022-12-17 15:44:22.552849: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Input Data Loading 

In [2]:
df_train = pd.read_csv('data/model_input/train_ultimatestrength.csv')
df_test = pd.read_csv('data/model_input/test_ultimatestrength.csv')
print('size of train data', len(df_train))
print('size of test data',len(df_test))


size of train data 117
size of test data 27


## Featurization Oliynyk

In [3]:
# FEATURIZATION 
X_train_unscaled, y_train, formulae_train, skipped_train = generate_features(df_train,elem_prop='oliynyk',drop_duplicates=False,extend_features=True)
X_test_unscaled, y_test, formulae_test, skipped_test = generate_features(df_test,elem_prop='oliynyk',drop_duplicates=False,extend_features=True)


scaler = StandardScaler()

X_train = scaler.fit_transform(X_train_unscaled)
X_test = scaler.transform(X_test_unscaled)
print(X_train.shape)
print(y_train.shape)


Processing Input Data: 100%|████████████████| 117/117 [00:00<00:00, 8910.28it/s]


	Featurizing Compositions...


Assigning Features...: 100%|████████████████| 117/117 [00:00<00:00, 6240.02it/s]


	Creating Pandas Objects...


Processing Input Data: 100%|██████████████████| 27/27 [00:00<00:00, 7627.03it/s]


	Featurizing Compositions...


Assigning Features...: 100%|██████████████████| 27/27 [00:00<00:00, 3780.29it/s]


	Creating Pandas Objects...
(117, 133)
(117,)


## Defining Helper Functions for model Performance Evaluation

In [4]:
################# EVALUATION MATRIX #######################################
def model_evaluation(y_actual,y_predict, label = None,model='None'):
    df = pd.DataFrame([{'mae':mean_absolute_error(y_actual,y_predict),
                        'rmse':mean_squared_error(y_actual,y_predict,squared=False),
                        'r2score':r2_score(y_actual,y_predict),
                        'model':model
                        }])
    if label:
        df.index = [label]
        
    return df

def model_evaluation_result(y_Train,y_train_predict,y_Test,y_test_predict,model='None'):
    result = pd.concat([model_evaluation(y_Train,y_train_predict,'Train',model),
                    model_evaluation(y_Test,y_test_predict, 'Test',model)])
    return result


def plot_model_performance(y_train,y_train_predict,y_test,y_test_predict,r2_train,r2_test):
    plt.figure(figsize=(8,8))
    plt.scatter(y_train,y_train_predict, s=160,alpha=0.7, label='Training Data ($r^2$ ={0:.3f})'.format(r2_train))
    plt.scatter(y_test,y_test_predict,s=160, alpha=0.7, label='Test Data ($r^2$ ={0:.3f})'.format(r2_test))
    # plt.plot(y_test,y_test,'--',dashes=(5, 6),linewidth=2,c='grey',label = 'Ideal Fit')
    plt.axline((0, 0), (1200, 1200), linewidth=2, color='k',linestyle='dashed',label='ideal fit')
    plt.xlabel('Actual Ultimate Tensile Strength (MPa) ', size=16)
    plt.ylabel('Predicted Ultimate Tensile Strength (MPa)',size=16)
    plt.tick_params(labelsize=14)
    plt.xlim((0,1200))
    plt.ylim((0,1200))
    plt.legend(prop={'size': 14})
    plt.minorticks_on()
    plt.tick_params(labelsize=16)
    plt.tick_params(direction='in',which='major', length=15, bottom=True, top=True, left=True, right=True,colors='k')
    plt.tick_params(direction='in',which='minor', length=5, bottom=True, top=True, left=True, right=True,colors='k')


## MLP with Grid Search CV

In [5]:
tensorflow.keras.utils.set_random_seed(20)


# Base estimator 
def ANN(optimizer = 'adam',neurons=32,batch_size=32,epochs=50,activation='relu',loss='mse'):
    model = Sequential()
    model.add(Dense(neurons, input_shape=(X_train.shape[1],), activation=activation))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(1))
    model.compile(optimizer = optimizer, loss=loss)
    return model

# Grid Search Param 
ann_params = {
    'optimizer': ['adam','rmsprop'],
    'activation': ['relu'],
    'loss': ['mse','mae'],
    'batch_size': [8,16],
    'neurons':[8,16,32],
    'epochs':[100,150,200,300],
}


# group K fold 5
gkf=GroupKFold(n_splits=5)

clf = KerasRegressor(build_fn=ANN, verbose=0)
grid = GridSearchCV(clf, ann_params,cv=gkf,scoring='neg_mean_squared_error')


grid.fit(X_train, y_train,groups=formulae_train)

print(grid.best_params_)

print("MSE:"+ str(-grid.best_score_))


best = grid.best_estimator_
tensorflow.keras.utils.set_random_seed(20)

best.fit(X_train,y_train)


/var/folders/k3/q1g0kjzs35sfzmwfr8ytcwkc0000gn/T/ipykernel_10372/1615911176.py:27: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  clf = KerasRegressor(build_fn=ANN, verbose=0)
2022-12-17 15:44:29.514754: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


{'activation': 'relu', 'batch_size': 8, 'epochs': 300, 'loss': 'mae', 'neurons': 32, 'optimizer': 'adam'}
MSE:44414.89662326318


## Performance Matrix

In [7]:
y_test_predict = best.predict(X_test)
y_train_predict = best.predict(X_train)
print(model_evaluation(y_train,y_train_predict))
print(model_evaluation(y_test,y_test_predict))


         mae       rmse   r2score model
0  32.929398  46.338743  0.936328  None
         mae        rmse   r2score model
0  96.910691  157.611855  0.624131  None


In [8]:
tensorflow.keras.utils.set_random_seed(20)

def create_model():
    model = Sequential()
    model.add(Dense(32,input_dim = X_train.shape[1],activation = 'relu'))
    
    model.add(Dense(32,activation = 'relu'))
    
    model.add(Dense(1))
    
    model.compile(loss = 'mae',optimizer = 'adam')
    return model

# Create the model

model = KerasRegressor(build_fn = create_model,verbose = 0,batch_size = 8,epochs = 300)

# Fitting the model

model.fit(X_train,y_train)

# Predicting using trained model

y_test_predict = model.predict(X_test)
y_train_predict = model.predict(X_train)
print(model_evaluation(y_train,y_train_predict))
print(model_evaluation(y_test,y_test_predict))

/var/folders/k3/q1g0kjzs35sfzmwfr8ytcwkc0000gn/T/ipykernel_10372/583384122.py:16: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(build_fn = create_model,verbose = 0,batch_size = 8,epochs = 300)


         mae       rmse   r2score model
0  32.929398  46.338743  0.936328  None
         mae        rmse   r2score model
0  96.910691  157.611855  0.624131  None


## Featurization with Custom Features 

In [9]:
# FEATURIZATION 
X_train_unscaled, y_train, formulae_train, skipped_train = generate_features(df_train,elem_prop='f3_revised',drop_duplicates=False,extend_features=True)
X_test_unscaled, y_test, formulae_test, skipped_test = generate_features(df_test,elem_prop='f3_revised',drop_duplicates=False,extend_features=True)


X_train_unscaled_avg = X_train_unscaled[['avg_Atomic_Radius','avg_Pauling_Electronegativity','avg_number_of_valence_electrons','avg_Cohesive_energy_ev_atom',
            'avg_Bulk_modulus_RT_Gpa','avg_Elastic_modulus_RT_Gpa','avg_Shear_modulus_RT_Gpa','avg_Melting_point_(K)','avg_rate_shear_mod_Mpa_perK',
            'avg_Solid_Solubility_atpct','avg_lattice_constant_A','avg_BEC_percm3','avg_Av.Valence_bond_strength_ev','avg_EngelZ_e/a','T']]

X_test_unscaled_avg = X_test_unscaled[['avg_Atomic_Radius','avg_Pauling_Electronegativity','avg_number_of_valence_electrons','avg_Cohesive_energy_ev_atom',
            'avg_Bulk_modulus_RT_Gpa','avg_Elastic_modulus_RT_Gpa','avg_Shear_modulus_RT_Gpa','avg_Melting_point_(K)','avg_rate_shear_mod_Mpa_perK',
            'avg_Solid_Solubility_atpct','avg_lattice_constant_A','avg_BEC_percm3','avg_Av.Valence_bond_strength_ev','avg_EngelZ_e/a','T']]



train_data_avg = X_train_unscaled_avg.join(y_train)
test_data_avg = X_test_unscaled_avg.join(y_test)
total_data_avg = pd.concat([train_data_avg,test_data_avg])
total_data_avg.shape


print('shape of the features space:',X_train_unscaled_avg.shape)


# Instantiate the standard scaler
scaler = StandardScaler()
# zero mean and unit variance
X_train = scaler.fit_transform(X_train_unscaled_avg)
X_test = scaler.transform(X_test_unscaled_avg)


Processing Input Data: 100%|███████████████| 117/117 [00:00<00:00, 19259.56it/s]


	Featurizing Compositions...


Assigning Features...: 100%|████████████████| 117/117 [00:00<00:00, 7731.98it/s]


	Creating Pandas Objects...


Processing Input Data: 100%|█████████████████| 27/27 [00:00<00:00, 11790.34it/s]


	Featurizing Compositions...


Assigning Features...: 100%|██████████████████| 27/27 [00:00<00:00, 3422.58it/s]

	Creating Pandas Objects...
shape of the features space: (117, 15)


## MLP wit Gird Search CV

In [10]:
tensorflow.keras.utils.set_random_seed(20)

def ANN(optimizer = 'adam',neurons=32,batch_size=32,epochs=50,activation='relu',loss='mse'):
    model = Sequential()
    model.add(Dense(neurons, input_shape=(X_train.shape[1],), activation=activation))
    model.add(Dense(neurons, activation=activation))
  
    model.add(Dense(1))
    model.compile(optimizer = optimizer, loss=loss)
    return model

ann_params = {
    'optimizer': ['adam','rmsprop'],
    'activation': ['relu'],
    'loss': ['mse','mae'],
    'batch_size': [8,16],
    'neurons':[8,16,32],
    'epochs':[100,150,200,300],
}


gkf=GroupKFold(n_splits=5)

clf = KerasRegressor(build_fn=ANN, verbose=0)
grid = GridSearchCV(clf, ann_params,cv=gkf,scoring='neg_mean_squared_error')


grid.fit(X_train, y_train,groups=formulae_train)

print(grid.best_params_)

print("MSE:"+ str(-grid.best_score_))


best = grid.best_estimator_
tensorflow.keras.utils.set_random_seed(20)

best.fit(X_train,y_train)


/var/folders/k3/q1g0kjzs35sfzmwfr8ytcwkc0000gn/T/ipykernel_10372/870770402.py:24: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  clf = KerasRegressor(build_fn=ANN, verbose=0)


{'activation': 'relu', 'batch_size': 8, 'epochs': 300, 'loss': 'mae', 'neurons': 32, 'optimizer': 'adam'}
MSE:25548.220434769628


## Perfomance Matrix

In [11]:
y_test_predict = best.predict(X_test)
y_train_predict = best.predict(X_train)
print(model_evaluation(y_train,y_train_predict))
model_evaluation(y_test,y_test_predict)

        mae       rmse   r2score model
0  40.10982  58.798253  0.897484  None


,mae,rmse,r2score,model
0,120.170009,155.936196,0.632081,None


In [12]:
tensorflow.keras.utils.set_random_seed(20)

def create_model():
    model = Sequential()
    model.add(Dense(32,input_dim = X_train.shape[1],activation = 'relu'))
    
    model.add(Dense(32,activation = 'relu'))
    
    model.add(Dense(1))
    
    model.compile(loss = 'mse',optimizer = 'adam')
    return model

# Create the model

model = KerasRegressor(build_fn = create_model,verbose = 0,batch_size = 8,epochs = 300)

# Fitting the model

model.fit(X_train,y_train)

# Predicting using trained model

y_test_predict = model.predict(X_test)
y_train_predict = model.predict(X_train)
print(model_evaluation(y_train,y_train_predict))
print(model_evaluation(y_test,y_test_predict))

/var/folders/k3/q1g0kjzs35sfzmwfr8ytcwkc0000gn/T/ipykernel_10372/2235905224.py:16: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(build_fn = create_model,verbose = 0,batch_size = 8,epochs = 300)


         mae       rmse   r2score model
0  38.799743  49.962906  0.925979  None
         mae        rmse   r2score model
0  133.92718  169.558215  0.564993  None


## Data Load for Composition 

In [13]:
df_train = pd.read_csv('data/model_input/train_ultimatestrength_composition.csv')
df_test = pd.read_csv('data/model_input/test_ultimatestrength_composition.csv')
print('size of train data', len(df_train))
print('size of test data',len(df_test))


size of train data 117
size of test data 27


In [14]:
X_train_unscaled = df_train[['Nb','Zr','V','Ta','W','Ti','Mo','C','Hf','T']]
y_train = df_train['target']
X_test_unscaled = df_test[['Nb','Zr','V','Ta','W','Ti','Mo','C','Hf','T']]
y_test = df_test['target']
formulae_train = df_train['formula']

In [15]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train_unscaled)
X_test = scaler.transform(X_test_unscaled)

X_test_unscaled.iloc[12]

Nb    87.7
Zr     1.1
V      0.0
Ta     0.0
W     11.2
Ti     0.0
Mo     0.0
C      0.0
Hf     0.0
T     24.0
Name: 12, dtype: float64

## MLP with Grid Search

In [16]:
tensorflow.keras.utils.set_random_seed(20)

def ANN(optimizer = 'adam',neurons=32,batch_size=32,epochs=50,activation='relu',loss='mse'):
    model = Sequential()
    model.add(Dense(neurons, input_shape=(X_train.shape[1],), activation=activation))
    model.add(Dense(neurons, activation=activation))
    
    model.add(Dense(1))
    model.compile(optimizer = optimizer, loss=loss)
    return model


ann_params = {
    'optimizer': ['adam','rmsprop'],
    'activation': ['relu'],
    'loss': ['mse','mae'],
    'batch_size': [8,16],
    'neurons':[8,16,32],
    'epochs':[100,150,200,300],
}





gkf=GroupKFold(n_splits=5)

clf = KerasRegressor(build_fn=ANN, verbose=0)
grid = GridSearchCV(clf, ann_params,cv=gkf,scoring='neg_mean_squared_error')


grid.fit(X_train, y_train,groups=formulae_train)

print(grid.best_params_)

print("MSE:"+ str(-grid.best_score_))


best = grid.best_estimator_
tensorflow.keras.utils.set_random_seed(20)


/var/folders/k3/q1g0kjzs35sfzmwfr8ytcwkc0000gn/T/ipykernel_10372/355730797.py:28: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  clf = KerasRegressor(build_fn=ANN, verbose=0)


{'activation': 'relu', 'batch_size': 8, 'epochs': 300, 'loss': 'mse', 'neurons': 16, 'optimizer': 'adam'}
MSE:20586.742201243538


## Performance Matrix

In [18]:
y_test_predict = best.predict(X_test)
y_train_predict = best.predict(X_train)
print(model_evaluation(y_train,y_train_predict))
print(model_evaluation(y_test,y_test_predict))

         mae       rmse   r2score model
0  39.671559  51.332061  0.921866  None
        mae        rmse   r2score model
0  101.6863  133.281931  0.731218  None
